### Fetching The 7z archive

In [ ]:
try:
    dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")
    print("Archive in filesystem (file:/databricks/driver/dblp.v13.7z)")
except:
    try:
        dbutils.fs.ls("dbfs:/FileStore/data/dblp.v13.7z")
        print("Archive located in FileStore. Copying into local store..")
        dbutils.fs.cp("dbfs:/FileStore/data/dblp.v13.7z", "file:/databricks/driver/dblp.v13.7z")
        print("Completed")
    except:
        print("7z archive not found. Fetching from URL...")
        !wget https://originalstatic.aminer.cn/misc/dblp.v13.7z
        print("7z archive Downloaded. Moving archive to FileStore..")
        dbutils.fs.mkdirs("dbfs:/FileStore/data")
        dbutils.fs.cp("file:/databricks/driver/dblp.v13.7z", "dbfs:/FileStore/data/dblp.v13.7z")
        print("Completed.")

In [ ]:
dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")

### Extracting Archive into json

**Skip this Section if you already have performed the extraction process.**

#### 1. Extraction

In [ ]:
!pip install py7zr -q

In [ ]:
import py7zr

archive = py7zr.SevenZipFile('dblp.v13.7z', mode='r')
archive.extractall()
archive.close()

In [ ]:
dbutils.fs.ls("file:/databricks/driver/dblpv13.json")

#### 2. Cleaning NumberInt(#) tags

In [ ]:
import re

# Cleaning the `NumberInt` tag
fin = open(f"dblpv13.json")
fout = open(f"dblpv13_clean.json", "wt")
for line in fin:
    fout.write(re.sub(r"NumberInt\([\d]*\)", lambda x: "".join(re.findall(r"\d", x.group(0))), line))
fin.close()
fout.close()

#### 3. Partitioning Dataset into JSON files

In [ ]:
!pip install ijson tqdm -q

In [ ]:
%mkdir data

In [ ]:
import ijson
import json
from tqdm import tqdm
import decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)

data_dir = 'data/'
with open('dblpv13_clean.json', 'r') as f:
    counter, file_id = 0, 0
    file_buffer = []
    for obj_data in ijson.items(f, 'item'):
        file_buffer.append(obj_data)
        counter += 1
        if counter % 300000 == 0:
            print(f" Saving, data_PART_{file_id}.json in {data_dir}")
            f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
            dump = json.dumps(file_buffer, cls=DecimalEncoder)
            f.write(dump)
            f.close()
            file_id += 1
            file_buffer = []
f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
dump = json.dumps(file_buffer, cls=DecimalEncoder)
print(f" Saving, data_PART_{file_id}.json in {data_dir}")
f.write(dump)
f.close()
file_id += 1
file_buffer = []

#### 4. Moving files to dbfs FileStore from instance storage, to make it available for later.

In [ ]:
# removing old json stored in filestore.
dbutils.fs.rm("dbfs:/FileStore/data/split_data/", recurse = True)
# Creating dir to store json in filestore..
dbutils.fs.mkdirs("dbfs:/FileStore/data/split_data")
# confirming dir is empty
dbutils.fs.ls("dbfs:/FileStore/data/split_data")

In [ ]:
# Copying all json parts into filestore.
dbutils.fs.cp("file:/databricks/driver/data/", "dbfs:/FileStore/data/split_data", recurse = True)

### Reading data from databricks Filestore into dataframes

In [ ]:
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, ArrayType  

path = "dbfs:/FileStore/data/split_data/"

# There should be 18 files each with 300 k records. This would change if you change split value.
file_count = len(dbutils.fs.ls(path))
assert file_count == 18, "Data not found. You may want to check the path or run the notebook from start again. If you updated the split value, ignore this assertion error"

In [ ]:
# https://stackoverflow.com/a/42540517 (This seems more proper way to merge the frames..)


# Currently Union fails as there seems to be difference between schemas in some instance in 9th Frame 

dataframes_map = map(lambda r: spark.read.option("inferSchema", True).json(r), [f"{path}data_PART_{num}.json" for num in range(file_count)])
union = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes_map)
union.printSchema()

# dataframes = []
# for num in range(file_count):
#    print(f"Reading file data_PART_{num}.json into spark")
#    dataframes.append(spark.read.option("inferSchema", True).json(f"{path}data_PART_{num}.json"))

In [ ]:
parquet_path = "/tmp/out"

def buildFoSTable(dataframe):
    # pull required Fields
    fosFrame = dataframe.select(F.explode_outer("fos").alias("Field_of_Study"))

    # fosFrame.show()
    # Clean (delete dups, Fill NaN values?, ...)
    fosFrame = fosFrame.fillna("Unknown")
    fosFrame = fosFrame.distinct()
    
    # Append to Parquet file
    # fos_frame.write.mode('append').parquet("/tmp/out/field_of_study.parquet")
    fosFrame.write.mode('overwrite').parquet(f"{parquet_path}/field_of_study.parquet")
    
    # pull appeneded parquet file and get distinct records
    fosFrame = spark.read.parquet("/tmp/out/field_of_study.parquet")
    
    # Index
    df = fosFrame.distinct()
    df = df.select("*").withColumn("id", F.monotonically_increasing_id())
    # return the indexed Table
    return df.select("id", "Field_of_Study")
    

In [ ]:
FoSFrame = buildFoSTable(union)
# map the relation in Fact Table
display(FoSFrame)

## PLAYGROUND/SANDBOX snippets

In [ ]:
path = "dbfs:/FileStore/data/split_data/"

first_frame = spark.read.option("inferSchema", True).json(f"{path}data_PART_0.json")
display(first_frame)

In [ ]:
first_frame.printSchema()

In [ ]:
authorsDF = first_frame.select(F.explode_outer("authors").alias("authors"))
authorsDF = authorsDF.selectExpr("authors._id", "authors.bio", "authors.email", "authors.gid", "authors.name", "authors.name_zh", "authors.oid", "authors.oid_zh", "authors.orcid", "authors.org", "authors.org_zh", "authors.orgid", "authors.orgs", "authors.orgs_zh", "authors.sid")

authorsDF.printSchema()

In [ ]:
display(authorsDF)

In [ ]:
# https://stackoverflow.com/questions/44831789/spark-sql-nested-withcolumn/44833112#44833112
import pyspark.sql.functions as F

first_frame = first_frame.select(F.explode_outer("authors").alias("a"), "*")
authors_df = first_frame.selectExpr("a._id", "a.bio", "a.email", "a.gid", "a.name", "a.name_zh", "a.oid", "a.oid_zh", "a.orcid", "a.org", "a.org_zh", "a.orgid", "a.orgs", "a.orgs_zh", "a.sid")

display(authors_df)

In [ ]:
display(first_frame)

In [ ]:
authors_df.printSchema()

In [ ]:
first_frame.printSchema()

In [ ]:
first_frame = first_frame.select(F.explode_outer("fos").alias("Field_of_Study"), "*")
fos_frame = first_frame.select(F.col("Field_of_Study"))

# fos_frame = first_frame.select(F.explode_outer("fos").alias("Field_of_Study"))
# fos_frame = fos_frame.select(F.col("Field_of_Study"))
display(fos_frame)

In [ ]:
fos_frame = fos_frame.distinct()
fos_frame.write.parquet("/tmp/out/field_of_study.parquet")
# fos_frame.write.mode('append').parquet("/tmp/out/field_of_study.parquet")

In [ ]:
parquet_path = "/tmp/out"

def buildFoSTable(dataframes):
    for frame in dataframes:
        # pull required Fields
        fosFrame = frame.select(F.explode_outer("fos").alias("Field_of_Study"))

        # fosFrame.show()
        # Clean (delete dups, Fill NaN values?, ...)
        fosFrame = fosFrame.fillna("Unknown")
        fosFrame = fosFrame.distinct()
        
        # Append to Parquet file
        # fos_frame.write.parquet("/tmp/out/field_of_study.parquet")
        fosFrame.write.mode('append').parquet(f"{parquet_path}/field_of_study.parquet")
    # pull appeneded parquet file and get distinct records
    fosFrame = spark.read.parquet("/tmp/out/field_of_study.parquet")
    # Index
    df = fosFrame.distinct()
    df = df.select("*").withColumn("id", F.monotonically_increasing_id())

    return df.select("id", "Field_of_Study")
    # save as table
    # map the relation in Fact Table


In [ ]:
# Merging Fact Tables..

df = fos_frame.distinct()
df = df.select("*").withColumn("id", F.monotonically_increasing_id())

display(df.select("id", "Field_of_Study"))

In [ ]:
first_frame.select("Field_of_Study").show()

In [ ]:
display(first_frame.filter(first_frame.Field_of_Study==df.Field_of_Study))

In [ ]:
# https://github.com/patelatharva/Data_Lake_with_Apache_Spark/blob/master/etl.py